<a href="https://colab.research.google.com/github/guymatz/csci-49376/blob/develop/csci_49376_project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
#drive.flush_and_unmount()

Mounted at /content/drive


In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 23.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=dcf1c97352892256d579c803c1b46f884411f897dac79bbef3813e53f8e2f359
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [3]:
# Import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql import functions
# Create a Spark Session
ss = SparkSession.builder.master("local[*]").getOrCreate()

In [36]:
edges = ss.read.csv("/content/drive/MyDrive/School/CSCI/Big-data/edges.tsv", header=True, sep="\t")
#edges = ss.read.csv("/content/drive/MyDrive/School/CSCI/Big-data/edges-test.tsv", header=True, sep="\t")
nodes = ss.read.csv("/content/drive/MyDrive/School/CSCI/Big-data/nodes.tsv", header=True, sep="\t")

In [37]:
print(f"Edges: {edges.count()}")
print(f"Nodes: {nodes.count()}")

Edges: 1292203
Nodes: 23036


Q1. 

For each drug, compute the number of genes
and the number of diseases associated with the
drug. Output results with top 5 number of genes in a descending orde

In [38]:
# Filter out the Compounds associated with Genes & Diseases
disease_compounds = edges.filter(
    edges.source.startswith('Compound') &
    (
        edges.target.startswith('Disease')
    )
)

gene_compounds = edges.filter(
    edges.source.startswith('Compound') &
    (
        edges.target.startswith('Gene') 
    )
)

In [39]:
gene_compounds.sample(.01).show(10)

+-----------------+--------+------------+
|           source|metaedge|      target|
+-----------------+--------+------------+
|Compound::DB01151|     CbG|  Gene::3358|
|Compound::DB00471|     CbG|  Gene::5742|
|Compound::DB00624|     CbG|  Gene::1588|
|Compound::DB00259|     CbG|   Gene::766|
|Compound::DB00604|     CbG|  Gene::1576|
|Compound::DB06708|     CbG|  Gene::1565|
|Compound::DB00675|     CbG|  Gene::1558|
|Compound::DB00674|     CbG|  Gene::1576|
|Compound::DB01577|     CbG|Gene::134864|
|Compound::DB04552|     CbG|  Gene::5742|
+-----------------+--------+------------+
only showing top 10 rows



In [40]:
disease_compounds.show(10)

+-----------------+--------+-------------------+
|           source|metaedge|             target|
+-----------------+--------+-------------------+
|Compound::DB00997|     CtD|  Disease::DOID:363|
|Compound::DB00206|     CtD|Disease::DOID:10763|
|Compound::DB00960|     CtD|Disease::DOID:10763|
|Compound::DB00665|     CtD|Disease::DOID:10283|
|Compound::DB00290|     CtD| Disease::DOID:2998|
|Compound::DB01232|     CtD|  Disease::DOID:635|
|Compound::DB00555|     CtD| Disease::DOID:1826|
|Compound::DB00444|     CtD| Disease::DOID:2531|
|Compound::DB00860|     CtD| Disease::DOID:4481|
|Compound::DB00635|     CtD| Disease::DOID:2531|
+-----------------+--------+-------------------+
only showing top 10 rows



In [41]:
# Convert  |source|target::xyz| -> |source|{target_type: 1}|
disease_rdd_1 = disease_compounds.rdd.map(lambda x: (x[0],  1) )
print("Diseases: " , disease_rdd_1.take(3))
gene_rdd_1 = gene_compounds.rdd.map(lambda x: (x[0],  1) )
print("Genes: ", gene_rdd_1.take(3))

Diseases:  [('Compound::DB00997', 1), ('Compound::DB00206', 1), ('Compound::DB00960', 1)]
Genes:  [('Compound::DB00514', 1), ('Compound::DB00686', 1), ('Compound::DB00786', 1)]


In [42]:
disease_rdd_2 = disease_rdd_1.reduceByKey( lambda x, y: x + y)
print("Diseases: " , disease_rdd_2.take(3))
gene_rdd_2 = gene_rdd_1.reduceByKey( lambda x, y: x + y)
print("Genes: " , gene_rdd_2.take(3))

Diseases:  [('Compound::DB00997', 17), ('Compound::DB00206', 1), ('Compound::DB00290', 6)]
Genes:  [('Compound::DB00514', 33), ('Compound::DB00786', 24), ('Compound::DB00217', 2)]


In [43]:
joined_rdd = gene_rdd_2.join(disease_rdd_2)
joined_rdd.take(5)

[('Compound::DB00514', (33, 2)),
 ('Compound::DB00203', (23, 3)),
 ('Compound::DB00909', (40, 1)),
 ('Compound::DB00575', (32, 6)),
 ('Compound::DB00502', (27, 4))]

In [44]:
joined_rdd.sortBy(lambda x: x[1][0], ascending = False).take(3)

[('Compound::DB08865', (585, 1)),
 ('Compound::DB01254', (564, 1)),
 ('Compound::DB00997', (532, 17))]

In [ ]:
# Q2: Compute the number of diseases associated
# with 1, 2, 3, …, n drugs. Output results with the top
# 5 number of diseases in a descending order.
disease_compounds = edges.filter(
    edges.source.startswith('Compound') &
    (
        edges.target.startswith('Disease')
    )

)

In [ ]:
# Q3: Get the name of drugs that have the top 5 number of genes. Output the results.